## Import libraries

In [1]:
import gc
import pickle
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_log_error

## Prepare data for model training

In [2]:
with open("../input/workation-price-prediction-preprocess-data-v2/Workation_Dataset.txt", 'rb') as handle: 
    data = handle.read()

processed_data = pickle.loads(data)
train_df = processed_data['train_df']
test_df = processed_data['test_df']

train_df = train_df[~((train_df['Per Person Price'] < 7.3) | (np.exp(train_df['Per Person Price']) > 150000))].copy()

del processed_data
gc.collect()

31

In [3]:
Xtrain = train_df.loc[:, train_df.columns != 'Per Person Price'].copy()
Ytrain = train_df['Per Person Price'].copy()
Ytrain_strat = pd.qcut(train_df['Per Person Price'].values, q=10, labels=range(0,10))
Xtest = test_df.copy()

print("Xtrain: {} \nYtrain: {} \nXtest: {}".format(Xtrain.shape, Ytrain.shape, Xtest.shape))

del train_df
del test_df
gc.collect()

Xtrain: (20989, 5752) 
Ytrain: (20989,) 
Xtest: (9000, 5752)


0

## Build and validate the model

In [4]:
FOLD = 10
NUM_SEED = 3

np.random.seed(3)
seeds = np.random.randint(0, 100, size=NUM_SEED)

oof_score = 0
y_pred_meta_cb = np.zeros((Ytrain.shape[0], NUM_SEED))
y_pred_final_cb = np.zeros((Xtest.shape[0], NUM_SEED))
counter = 0


for sidx, seed in enumerate(seeds):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain.values, Ytrain_strat)):
        counter += 1

        train_x, train_y = Xtrain.iloc[train], Ytrain.iloc[train]
        val_x, val_y = Xtrain.iloc[val], Ytrain.iloc[val]

        model = CatBoostRegressor(
            objective='RMSE',
            eval_metric='RMSE',
            num_boost_round=8000,
            learning_rate=0.01,
            reg_lambda=0.0432,
            bootstrap_type='Poisson',
            subsample=0.96,
            max_depth=12, 
            grow_policy='Lossguide',
            min_data_in_leaf=2, 
            max_leaves=65,
            task_type='GPU',
            verbose=0
        )

        model.fit(train_x, train_y, eval_set=[(val_x, val_y)], 
                  early_stopping_rounds=200, verbose=100)

        y_pred = model.predict(val_x)
        y_pred_meta_cb[val, sidx] += y_pred
        y_pred_final_cb[:, sidx] += model.predict(Xtest)
        
        score = np.sqrt(mean_squared_log_error(val_y, y_pred))
        oof_score += score
        seed_score += score
        print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_final_cb = y_pred_final_cb / float(FOLD)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

0:	learn: 0.5364238	test: 0.5317242	best: 0.5317242 (0)	total: 90.9ms	remaining: 12m 7s
100:	learn: 0.2823726	test: 0.2850707	best: 0.2850707 (100)	total: 5.94s	remaining: 7m 44s
200:	learn: 0.1985328	test: 0.2121340	best: 0.2121340 (200)	total: 12s	remaining: 7m 46s
300:	learn: 0.1657146	test: 0.1886679	best: 0.1886679 (300)	total: 17.8s	remaining: 7m 35s
400:	learn: 0.1488884	test: 0.1794604	best: 0.1794604 (400)	total: 24.3s	remaining: 7m 40s
500:	learn: 0.1382366	test: 0.1746251	best: 0.1746251 (500)	total: 29.6s	remaining: 7m 23s
600:	learn: 0.1308674	test: 0.1716255	best: 0.1716255 (600)	total: 35s	remaining: 7m 10s
700:	learn: 0.1251861	test: 0.1697360	best: 0.1697360 (700)	total: 39.9s	remaining: 6m 55s
800:	learn: 0.1205294	test: 0.1685623	best: 0.1685623 (800)	total: 45s	remaining: 6m 44s
900:	learn: 0.1166057	test: 0.1676125	best: 0.1676125 (900)	total: 49.9s	remaining: 6m 33s
1000:	learn: 0.1130596	test: 0.1667948	best: 0.1667948 (1000)	total: 56s	remaining: 6m 31s
1100:	le

In [5]:
np.savez_compressed('./CB_Meta_Features.npz',
                    y_pred_meta_cb=y_pred_meta_cb, 
                    oof_score=oof_score,
                    y_pred_final_cb=y_pred_final_cb)

In [6]:
df = pd.DataFrame(y_pred_final_cb)
df.head()

,0,1,2
0,9.856073,9.856940,9.847776
1,9.369113,9.373182,9.368874
2,8.655177,8.652033,8.659614
3,8.860264,8.860241,8.857832
4,9.983650,9.984871,9.985201
